## <center>Capstone Project </center>

<center> 18th March </center>

Group Team
- Ethan Lindt
- Andrew Olsen
- Alessandro Caruso

<hr>

<p style="background-color: darkgreen;"><strong>Done:</strong>&nbsp;</p>


- Main exploration on c110 (called df110)
- Data Cleaning
- Exploration of main variables
- Creation of new features
- Approximation of ISI lead time
- Inspection of relationships between regressors and waiting time 
- Initial definition of an efficient delivery
- Grouping of product and suppliers of which the ordered quantity has an impact on waiting time.
- preliminary linear regression to assess impact of coefficients

<p style="background-color: navy;"><strong>In Progress:</strong>&nbsp;</p>

- Definition of efficient delivery
- ad-hoc cleaning and feature engineering
- Exploration of df126

<p style="background-color: darkred;"><strong>Next Steps:</strong>&nbsp;</p>

- Cross-Analysis with the other datasets
- Predictive models
- What-if scenarios
- Comprehensive and Interactive Data visualizations


## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycodestyle
import openpyxl

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('magma')

In [2]:
df_110 = pd.read_excel('USD PO Line Detail Report.xlsx', sheet_name = 0, engine = 'openpyxl')
df_126 = pd.read_excel('USD PO Line Detail Report.xlsx', sheet_name = 1, engine = 'openpyxl')
df_297 = pd.read_excel('USD PO Line Detail Report.xlsx', sheet_name = 3, engine = 'openpyxl')
df_298 = pd.read_excel('USD PO Line Detail Report.xlsx', sheet_name = 4, engine = 'openpyxl')

## Data Cleaning

In [3]:
df_110 = df_110[~df_110.iloc[:,0].str.startswith('1')].drop(columns= ['Confirmed Delivery Date'], axis = 1)
df_126 = df_126[~df_126.iloc[:,0].str.startswith('1')].drop(columns= ['Confirmed Delivery Date'], axis = 1)
df_297 = df_297[~df_297.iloc[:,0].str.startswith('1')].drop(columns= ['Confirmed Delivery Date'], axis = 1)
df_298 = df_298[~df_298.iloc[:,0].str.startswith('1')].drop(columns= ['Confirmed Delivery Date'], axis = 1)


df_110 = df_110[~df_110['Actual Delivery Date'].isna()]
df_126 = df_126[~df_126['Actual Delivery Date'].isna()]
df_297 = df_297[~df_297['Actual Delivery Date'].isna()]
df_298 = df_298[~df_298['Actual Delivery Date'].isna()]


df_110['Actual Delivery Date'] = df_110['Actual Delivery Date'].values.astype('datetime64[D]')
df_126['Actual Delivery Date'] = df_126['Actual Delivery Date'].values.astype('datetime64[D]')
df_297['Actual Delivery Date'] = df_297['Actual Delivery Date'].values.astype('datetime64[D]')
df_298['Actual Delivery Date'] = df_298['Actual Delivery Date'].values.astype('datetime64[D]')


df_110['PO Creation Date'] = df_110['PO Creation Date'].values.astype('datetime64[D]')
df_126['PO Creation Date'] = df_126['PO Creation Date'].values.astype('datetime64[D]')
df_297['PO Creation Date'] = df_297['PO Creation Date'].values.astype('datetime64[D]')
df_298['PO Creation Date'] = df_298['PO Creation Date'].values.astype('datetime64[D]')



### Dropping Products with Same Order Quantity

In [4]:
df_110 = df_110.rename(columns={"Part Number": "Part_Number"})
df_110_unique = df_110.groupby(['Part_Number', 'BP Code'])['Ordered Qty'].nunique().reset_index(name='count')

df_110_unique.Part_Number = df_110_unique.Part_Number.astype('str')
df_110_unique['BP Code'] = df_110_unique['BP Code'].astype('str')
df_110_unique['PN_BP_code'] = df_110_unique.Part_Number + '_'+ df_110_unique['BP Code']
df_110_unique = df_110_unique[df_110_unique['count'] == 1].reset_index(drop = True)

df_110.Part_Number = df_110.Part_Number.astype('str')
df_110['BP Code'] = df_110['BP Code'].astype('str')
df_110['PN_BP_code'] = df_110.Part_Number + '_' + df_110['BP Code']

In [5]:
df_110 = df_110[~df_110.PN_BP_code.isin(df_110_unique.PN_BP_code)].reset_index(drop = True)

In [6]:
#drop now but we'll have to change these 181 entries later as we don't have the price.
df_110 = df_110[~df_110['Unit Price'].isna()].reset_index(drop = True)
df_126 = df_126[~df_126['Unit Price'].isna()].reset_index(drop = True)
df_297 = df_297[~df_297['Unit Price'].isna()].reset_index(drop = True)
df_298 = df_298[~df_298['Unit Price'].isna()].reset_index(drop = True)

## (Initial) Features Engineering

In [7]:
df_110['true_dem_lt'] = df_110.apply(lambda x: np.busday_count(x['PO Creation Date'].date(), x['Actual Delivery Date'].date()), axis = 1)

df_110['quantity_not_delivered'] = df_110['Ordered Qty'] - df_110['Delivered Qty']

df_110['Week_late'] = pd.cut(df_110.loc[:,'true_dem_lt'], bins = list(np.arange(7,70,7, dtype = int)), labels= False, right = True, include_lowest= True)
df_110.Week_late.fillna('8', inplace= True)

df_110

,PO Number,POS Number,Seq Number,BP Code,Part_Number,Ordered Qty,Delivered Qty,ISI Lead Time,Dem LT,PO Creation Date,Actual Delivery Date,Unit Price,PN_BP_code,true_dem_lt,quantity_not_delivered,Week_late
0,BAP000003,10,0,SLD063461,233313-8-0V,1.0,1.0,NaN,8.0,2020-10-19,2020-10-29,2718.31,233313-8-0V_SLD063461,8,0.0,0
1,BAP000006,10,0,SUP067572,204286-4-2V,3.0,3.0,5.0,4.0,2020-10-27,2020-11-02,62.50,204286-4-2V_SUP067572,4,0.0,8
2,BAP000007,10,0,SUP034822,219636-1,19.0,16.0,130.0,169.0,2020-11-11,2021-07-02,1746.00,219636-1_SUP034822,167,3.0,8
3,BAP000007,10,1,SUP034822,219636-1,3.0,2.0,130.0,169.0,2020-11-11,2021-07-21,1746.00,219636-1_SUP034822,180,1.0,8
4,BAP000008,10,0,SUP034822,219636-1,19.0,18.0,130.0,169.0,2020-11-11,2021-07-02,1746.00,219636-1_SUP034822,167,1.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152851,TSS009928,10,0,SUP049443,23819R1,200.0,200.0,7.0,7.0,2021-12-14,2021-12-23,2.72,23819R1_SUP049443,7,0.0,0
152852,TSS009930,10,0,SUP049443,21535R1,40.0,40.0,7.0,7.0,2021-12-14,2021-12-23,0.03,21535R1_SUP049443,7,0.0,0
152853,TSS009930,20,0,SUP049443,1007238,50.0,50.0,7.0,7.0,2021-12-14,2021-12-23,1.54,1007238_SUP049443,7,0.0,0
152854,TSS009930,30,0,SUP049443,701643C1,100.0,100.0,7.0,7.0,2021-12-14,2021-12-23,0.06,701643C1_SUP049443,7,0.0,0


In [8]:
def _rename_df(df):
    col = ['Week_Late_'+str(i)+'.0' for i in range(0,8)]
    for j in range(len(col)):
        df = df.rename({col[j]:col[j][:-2]}, axis = 1)
    return df

df_110 = df_110.merge(pd.get_dummies(df_110.Week_late, prefix= 'Week_Late'), left_index= True, right_index= True)

df_110 = _rename_df(df_110)

In [9]:
df_110.columns

Index(['PO Number', 'POS Number', 'Seq Number', 'BP Code', 'Part_Number',
       'Ordered Qty', 'Delivered Qty', 'ISI Lead Time', 'Dem LT',
       'PO Creation Date', 'Actual Delivery Date', 'Unit Price', 'PN_BP_code',
       'true_dem_lt', 'quantity_not_delivered', 'Week_late', 'Week_Late_0',
       'Week_Late_1', 'Week_Late_2', 'Week_Late_3', 'Week_Late_4',
       'Week_Late_5', 'Week_Late_6', 'Week_Late_7', 'Week_Late_8'],
      dtype='object')

In [10]:
df_126['true_dem_lt'] = df_126.apply(lambda x: np.busday_count(x['PO Creation Date'].date(), x['Actual Delivery Date'].date()), axis = 1)

df_126['quantity_not_delivered'] = df_126['Ordered Qty'] - df_126['Delivered Qty']



In [11]:
df_297['true_dem_lt'] = df_297.apply(lambda x: np.busday_count(x['PO Creation Date'].date(), x['Actual Delivery Date'].date()), axis = 1)

df_297['quantity_not_delivered'] = df_297['Ordered Qty'] - df_297['Delivered Qty']



In [12]:
df_298['true_dem_lt'] = df_298.apply(lambda x: np.busday_count(x['PO Creation Date'].date(), x['Actual Delivery Date'].date()), axis = 1)

df_298['quantity_not_delivered'] = df_298['Ordered Qty'] - df_298['Delivered Qty']




## Data Exploration

We first start our analysis by understanding the dataset at hand. 
<br> Our main objective is to improve delivery performance by optimizing the size of the order: the order quantity. 
<br> We define an order as
- <strong> Efficient</strong>: if the supplier was able to deliver the ordered quantity within the ISI lead time (the estimated number of days given by the supplier to fulfil the order).
- <strong>  Inefficient</strong>: otherwise.

<br> If the ISI Lead Time has not been disclosed by the supplier, we estimate it. 

### Approximation of Lead Time

Lead time is the amount of days that a supplier will take in order to deliver a specific product, products may be delivered together and they take different time according to the quantity ordered. For the sake of simplicity we approximate the lead time for those orders that lack one, by taking the mean lead time asked to deliver a particular order both by product and by supplier. 

In [13]:
def avg_lt_by(dt, by_what = 'Part Number'):
    """
    This function approximates lead time, by taking the average lead time for that specific product or for that specific supplier
    """
    df = dt.copy()
    prod_med_lt = df.groupby(by_what)[['ISI Lead Time']].median().reset_index().rename(columns = {'ISI Lead Time':'median_lead_time'})
    prod_med_lt = prod_med_lt.set_index(by_what).to_dict()['median_lead_time']
    if 'ISI_lead_time' not in dt.columns: 
        df['isi_lt_missing'] = df['ISI Lead Time'].isna()
        dt['ISI_lead_time'] = df.apply(lambda x: prod_med_lt[x[by_what]] if x.isi_lt_missing == True else x['ISI Lead Time'], axis = 1)
    else: 
        df['isi_lt_missing'] = dt['ISI_lead_time'].isna()
        dt['ISI_lead_time'] = df.apply(lambda x: prod_med_lt[x[by_what]] if x.isi_lt_missing == True else x['ISI_lead_time'], axis = 1)


In [14]:
avg_lt_by(df_110, by_what = 'BP Code')
avg_lt_by(df_110)

KeyError: 'Part Number'

In [ ]:
print(f"We still have {df_110.ISI_lead_time.isna().sum()} rows where ISI lead time is not present, or has not been approximated so we drop this.")
df_110 = df_110[~df_110.ISI_lead_time.isna()].reset_index(drop = True)

We still have 4317 rows where ISI lead time is not present, or has not been approximated so we drop this.


In [ ]:
#Deal with outliers, there are some datapoints where the Actual Delivery date is larger than PO Creation Date, we filter those rows out.
df_110 = df_110[df_110.true_dem_lt >= 0].reset_index(drop = True)

We now try to come up with the first draft of an efficiency measure: namely if the supplier was able to deliver within its disclosed estimated lead time, then we will classify that deliver as efficient, otherwise not.

In [311]:
def compute_efficiency(dt):
    df = dt.copy()
    df['efficient_shipment'] = df.apply(lambda x: 0 if x.true_dem_lt > x.ISI_lead_time else 1, axis = 1)
    return df

In [4]:
df_110.columns

Index(['PO Number', 'POS Number', 'Seq Number', 'BP Code', 'Part Number',
       'Ordered Qty', 'Delivered Qty', 'ISI Lead Time', 'Dem LT',
       'PO Creation Date', 'Confirmed Delivery Date', 'Actual Delivery Date',
       'Unit Price'],
      dtype='object')

In [88]:
import statsmodels.api as sm

def linear_est_lt(df, supplier):
    """
    This function runs a linear regression for the selected supplier and outputs the coefficient of quantity and the products supplied.
    The linear estimation function equals:
    lead_time ~ const + b1*Quantity + b2 * p1 + ... + bn*pn

    :df pandas dataframe to be used
    :supplier string, supplier code
    """
    assert isinstance(supplier, str) == True
    
    df_by_supplier = df[df.loc[:,'BP Code'] == supplier][['Ordered Qty','Part Number','Dem LT']].reset_index(drop = True)
    
    try:
        assert df_by_supplier.shape[0] > 10
        df_by_supplier = pd.get_dummies(df_by_supplier, drop_first = True)

        X = df_by_supplier.drop(columns= ['Dem LT'], axis = 1)
        X = sm.add_constant(X)
        y = df_by_supplier.loc[:,'Dem LT']

        mod = sm.OLS(y,X)
        res = mod.fit()

        print(res.summary())
        est_param = res.params.to_dict()
        return supplier, est_param
    except:
        print(f"Not enough datapoints, only {df_by_supplier.shape[0]} for supplier {supplier}")
        return supplier, np.zeros()


In [89]:
suppliers = df_110['BP Code'].unique()
for sup in suppliers:
    linear_est_lt(df_110, sup)
    

                            OLS Regression Results                            
Dep. Variable:                 Dem LT   R-squared:                       0.462
Model:                            OLS   Adj. R-squared:                  0.279
Method:                 Least Squares   F-statistic:                     2.527
Date:                Tue, 29 Mar 2022   Prob (F-statistic):            0.00569
Time:                        16:21:59   Log-Likelihood:                -236.95
No. Observations:                  68   AIC:                             509.9
Df Residuals:                      50   BIC:                             549.9
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

C:\Users\xelin\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


                            OLS Regression Results                            
Dep. Variable:                 Dem LT   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                 -0.705
Method:                 Least Squares   F-statistic:                    0.3207
Date:                Tue, 29 Mar 2022   Prob (F-statistic):              0.997
Time:                        16:22:36   Log-Likelihood:                -267.72
No. Observations:                  47   AIC:                             593.4
Df Residuals:                      18   BIC:                             647.1
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

C:\Users\xelin\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\xelin\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


                            OLS Regression Results                            
Dep. Variable:                 Dem LT   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     19.88
Date:                Tue, 29 Mar 2022   Prob (F-statistic):           6.00e-05
Time:                        16:22:38   Log-Likelihood:                -58.445
No. Observations:                  16   AIC:                             124.9
Df Residuals:                      12   BIC:                             128.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [54]:
X.drop(columns= ['const','Ordered Qty'], axis =1)

,Part Number_SV_1706513,Part Number_SV_1706514,Part Number_SV_2072071,Part Number_SV_2072072,Part Number_SV_2072073,Part Number_SV_2153542,Part Number_SV_2153543,Part Number_SV_2153544,Part Number_SV_2153545,Part Number_SV_2153548,...,Part Number_SV_3241696,Part Number_SV_3241735,Part Number_SV_3241770,Part Number_SV_3241771,Part Number_SV_3241772,Part Number_SV_3241774,Part Number_SV_3241777,Part Number_SV_3241778,Part Number_SV_5612253,Part Number_SV_5612261
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
 import statsmodels.api as sm

X = test.drop(columns= ['Dem LT'], axis = 1)
X = sm.add_constant(X)
y = test.loc[:,'Dem LT']

mod = sm.OLS(y,X)

res = mod.fit()
res.summary()
res.params.to_dict()



In [ ]:
def predict(X, Y, Z, df_110):
    mean_X = np.mean(X)
    mean_Y = np.mean(Y)
    mean_Z = np.mean(Z)
    beta2 = sum([(Z[i] - mean_Z)*(Y[i] - mean_Y) for i in range(len(Z))]) / sum([(Z[i] - mean_Z)**2 for i in range(len(Z))])
    beta1 = sum([(X[i] - mean_X)*(Y[i] - mean_Y) for i in range(len(X))]) / sum([(X[i] - mean_X)**2 for i in range(len(X))])
    beta0 = mean_Y - beta1 * mean_X - beta2 * mean_Z 
    y_hat = [beta0 + beta1*X[i] for i in range(len(X))]
    df_110['prediction'] = y_hat
    return df_110